In [1]:
import sys
sys.path.append('../')
from optm_algorithms.pso import PSO
from optm_algorithms.differential_evolution import DifferentialEvolutionAlgorithm

from fitness_function import RocketFitness, bound_values, fitness_func
import numpy as np
import warnings
from tqdm import tqdm
import json


In [2]:
# Inicializando função de fitness
rocket_fitness = RocketFitness(bound_values, num_workers=2)
random_values = np.random.rand(10,10)
fitness_func_class = rocket_fitness.calc_fitness


# Análise de Sensibilidade do PSO

Hiperparâmetros a serem otimizados:
- global_factor
- speed_factor
- v_max

Condições:
- local_factor + global_factor > 4
- todos os hiperparâmetros são maiores que zero

In [3]:
global_factor_list = np.linspace(1, 6, 5)
local_factor_list = np.linspace(1, 6, 5)
v_max = np.linspace(1,10,5)
grid1, grid2, grid3 = np.meshgrid(global_factor_list, local_factor_list, v_max)

combinations = np.vstack((grid1.ravel(), grid2.ravel(), grid3.ravel())).T
sum_of_columns = combinations[:, 0] + combinations[:, 1]

# Use logical indexing to select rows where the sum of the first and second columns is greater than 4
combinations = combinations[sum_of_columns > 4]


In [4]:
combinations.shape

(110, 3)

In [5]:

def execute_sensitivity_analysis_pso(combinations, filename):
    
    #combinations = [[2.05, 2.05, 1],[2.05, 2.05, 1]]

    simulations_list = []
    for row in tqdm(combinations):
        
        global_factor = row[0]
        local_factor = row[1]
        v_max = row[2]
        
        pso_alg = PSO(
            num_epochs=100,
            pop_size=1000,
            chrom_length=10,
            n_best=2,
            global_factor = global_factor,
            local_factor = local_factor,
            speed_factor = 1,
            v_max=v_max,
            value_ranges=bound_values,
            fitness_func=fitness_func_class,
            neighborhood_mode='self',
            verbose=True,
            eval_every=100,
            )

        best_solutions = pso_alg.fit()

        dict_save = {'global_factor': global_factor,
                     'local_factor': local_factor,
                     'v_max': v_max,
                     'fitness_calls': pso_alg.fitness_calls_list.tolist(),
                     'best_ind_list': pso_alg.best_ind_list.tolist(),
                     'avg_ind_list': pso_alg.avg_ind_list.tolist(),
                     'best_solutions': best_solutions.tolist(),
                     'total_time': pso_alg.total_exec_time,
                     }
        simulations_list.append(dict_save)
        
    with open(filename, 'w') as fout:
        json.dump(simulations_list, fout)

    return
        

execute_sensitivity_analysis_pso(combinations, 'simulations/pso_sensitivity.json')

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch 0: Best: 0.5688369478323626, Average: 0.21585542982278935


  1%|          | 1/110 [04:16<7:46:04, 256.56s/it]

--- 256.55598187446594 seconds ---


Epoch 0: Best: 0.5688369478323626, Average: 0.21585542982278935


...

 99%|█████████▉| 109/110 [9:29:31<04:24, 264.41s/it]

--- 240.65847897529602 seconds ---


Epoch 0: Best: 0.5688369478323626, Average: 0.21585542982278935


100%|██████████| 110/110 [9:33:32<00:00, 312.84s/it]


--- 240.6121551990509 seconds ---


# Análise de Sensibilidade da Evol ução Diferencial

Hiperparâmetros a serem otimizados:
- mutation_rate
- crossover_rate

Condições:
- mutation_rate e crossover_rate <=1


In [6]:
mutation_rate = np.linspace(0.1, 1, 10)
crossover_rate = np.linspace(0.1, 1,10)
grid1, grid2 = np.meshgrid(mutation_rate, crossover_rate)
combinations = np.vstack((grid1.ravel(), grid2.ravel())).T

In [8]:

def execute_sensitivity_analysis_de(combinations, filename):
    
    #combinations = [[0.5, 0,5]]

    simulations_list = []
    for row in tqdm(combinations):
        
        mutation_rate = row[0]
        crossover_rate = row[1]
        
        de_alg = DifferentialEvolutionAlgorithm(
                                            num_epochs=100,
                                            pop_size=1000,
                                            chrom_length=10,
                                            value_ranges=bound_values,
                                            mutation_rate=mutation_rate,
                                            crossover_rate=crossover_rate,
                                            fitness_func=fitness_func_class,
                                            verbose=True,
                                            eval_every=100,
                                            )

        best_solutions = de_alg.fit()

        dict_save = {'global_factor': mutation_rate,
                     'local_factor': crossover_rate,
                     'fitness_calls': de_alg.fitness_calls_list.tolist(),
                     'best_ind_list': de_alg.best_ind_list.tolist(),
                     'avg_ind_list': de_alg.avg_ind_list.tolist(),
                     'best_solutions': best_solutions.tolist(),
                     'total_time': de_alg.total_exec_time,
                     }
        simulations_list.append(dict_save)
        
    with open(filename, 'w') as fout:
        json.dump(simulations_list, fout)

    return
        

execute_sensitivity_analysis_de(combinations, 'simulations/de_sensitivity.json')

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0: Best: 0.5688369478323626, Average: 0.24990544051605962


  1%|          | 1/100 [08:11<13:31:39, 491.91s/it]

--- 491.9082601070404 seconds ---


Epoch 0: Best: 0.5688369478323626, Average: 0.2730744545038224


  2%|▏         | 2/100 [30:33<26:59:40, 991.64s/it]

--- 1341.4521889686584 seconds ---


  2%|▏         | 2/100 [30:36<24:59:58, 918.35s/it]


KeyboardInterrupt: 